In [12]:
import numpy as np

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
import scipy.spatial.distance as spdist

def pass_arg(Xx, nsim, tr_size):

    print("tr_Size:",tr_size)
    
    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))
    
    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn
    
    # Experimental data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = int(tr_size)

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]


    # Physics data
    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    y_unlabeled1 = data_phyloss[:1303, -2:-1]
    y_unlabeled2 = data_phyloss[-6:, -2:-1]

    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))
    y_unlabeled = np.vstack((y_unlabeled1,y_unlabeled2))

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_unlabeled = scaler.fit_transform(x_unlabeled)

    # initial porosity
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))


    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''

        [n, D] = x.shape
        ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
        sf2 = np.array(hyp[D])**2         # signal variance
        tmp = np.dot(np.diag(ell),x.T).T
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances
        A = sf2*np.exp(-0.5*A)  

        return A


    def posterior_predictive(X_s, X_train, Y_train, l1=.1, l2=.1, sigma_f=.1, sigma_y=0):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_train) + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                noise**2 * np.eye(len(X_train))
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
            return np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)

        if naive:
            return nll_naive
        else:
            return nll_stable

        
    def posterior_predictive_discp(X_s, X_train, Y_train, l1=.1, l2=.1, sigma_f=.1, sigma_y=1e-2):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_train) + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn_discp(X_train, Y_train, x_unlabeled, init_poro, phy_pred, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable_discp(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                theta[3]**2 * np.eye(len(X_train))
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
#             return np.sum(np.log(np.diagonal(L))) + \
#                    0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
#                    0.5 * len(X_train) * np.log(2*np.pi)
        
            mu_discp, _ = posterior_predictive_discp(x_unlabeled, X_train, Y_train, l1=theta[0], l2=theta[1], sigma_f=theta[2], sigma_y=theta[3])
            
            # physics predictions GP1+GP2 (for physics inputs)
            mu_un = phy_pred + mu_discp
            phyloss_poro = np.mean(poros(init_poro, mu_un))

            log_loss = np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)
        
            print(500000*phyloss_poro,log_loss, theta)
            return 500000*phyloss_poro + log_loss
        
        

        return nll_stable_discp

        
    
    # Optimization - GP1 - for physics
    res1 = minimize(nll_fn(x_unlabeled, y_unlabeled), x0 = [.1, .1, .1], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None)),
                    method='L-BFGS-B')
    
    # GP (physics) predictions for the physics sets
    mu_phy, cov_phy = posterior_predictive(x_unlabeled, x_unlabeled, y_unlabeled, *res1.x)
    
    # GP (physics) predictions for the experimental sets
    mu_s1, cov_s1 = posterior_predictive(trainX, x_unlabeled, y_unlabeled, *res1.x)
    
    # discrepancy
    discp = trainY - mu_s1
    
    # Optimization - GP2 - for discrepency
    res2 = minimize(nll_fn_discp(trainX, discp, x_unlabeled, init_poro, mu_phy), x0 = [.1, .1, .1, 1e-3], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None), (1e-7, None)),
                    method='L-BFGS-B')
    
    # GP (physics) predictions
    mu_s2, cov_s2 = posterior_predictive_discp(trainX, trainX, discp, *res2.x)
    
#     print(mu_s2+mu_s1, discp)
    pred_mu = mu_s2+mu_s1
    pred_cov = cov_s2+cov_s1
    samples = np.random.multivariate_normal(pred_mu.ravel(), pred_cov, int(nsim))
    
#     print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
#     print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    
    
    
    print("RMSE:", root_mean_squared_error(trainY, pred_mu))

    return samples, trainY, res1, res2, mu_s1, mu_s2, cov_s1, cov_s2

In [13]:
Xx = np.random.uniform(size=(5, 2))
samples, trainY, res1, res2, mu_s1, mu_s2, cov_s1, cov_s2 = pass_arg(Xx, 1, 39)
# print(ss)

tr_Size: 39


<ipython-input-12-49e833467f3a>:150: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
<ipython-input-12-49e833467f3a>:227: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \


1.750190069504567 [[37.45571262]] [0.1   0.1   0.1   0.001]
1.7501924239502944 [[37.45571204]] [0.10000001 0.1        0.1        0.001     ]
1.7501979158724328 [[37.45571169]] [0.1        0.10000001 0.1        0.001     ]
1.7501900746938843 [[37.45571256]] [0.1        0.1        0.10000001 0.001     ]
1.7501895376947723 [[37.45421436]] [0.1        0.1        0.1        0.00100001]
0.0 [[36.36551902]] [0.09999933 0.09999933 0.10003464 1.001     ]
0.0 [[36.36551902]] [0.09999934 0.09999933 0.10003464 1.001     ]
0.0 [[36.36551902]] [0.09999933 0.09999934 0.10003464 1.001     ]
0.0 [[36.36551906]] [0.09999933 0.09999933 0.10003465 1.001     ]
0.0 [[36.3655194]] [0.09999933 0.09999933 0.10003464 1.00100001]
0.0 [[-3.28477917]] [0.09999978 0.09999978 0.10001153 0.33388853]
0.0 [[-3.28477919]] [0.09999979 0.09999978 0.10001153 0.33388853]
0.0 [[-3.2847792]] [0.09999978 0.09999979 0.10001153 0.33388853]
0.0 [[-3.28477899]] [0.09999978 0.09999978 0.10001154 0.33388853]
0.0 [[-3.28477818]] [0.0

6.873217315704928 [[-81.31745882]] [0.1129083  0.09301034 0.08359889 0.00947376]
6.873209087141621 [[-81.31745928]] [0.11290829 0.09301035 0.08359889 0.00947376]
6.873208839965995 [[-81.31745919]] [0.11290829 0.09301034 0.0835989  0.00947376]
6.873195822261202 [[-81.31744544]] [0.11290829 0.09301034 0.08359889 0.00947377]
10.111350148222225 [[191.87533735]] [1.10213895e-01 9.05699062e-02 8.00108182e-02 1.00000000e-07]
10.145441238148996 [[191.8753366]] [1.10213905e-01 9.05699062e-02 8.00108182e-02 1.00000000e-07]
10.127196305507459 [[191.87533609]] [1.10213895e-01 9.05699162e-02 8.00108182e-02 1.00000000e-07]
10.13949534374293 [[191.87533581]] [1.10213895e-01 9.05699062e-02 8.00108282e-02 1.00000000e-07]
10.131440409004174 [[191.8753367]] [1.10213895e-01 9.05699062e-02 8.00108182e-02 1.10000000e-07]
6.8735227441686435 [[-81.31792226]] [0.11290818 0.09301023 0.08359875 0.00947336]
6.873532545032196 [[-81.31792295]] [0.11290819 0.09301023 0.08359875 0.00947336]
6.873524316400484 [[-81.31

0.9788975688621342 [[-84.838624]] [0.10223909 0.09947458 0.08391913 0.00580692]
0.9789010391025479 [[-84.83862473]] [0.1022391  0.09947458 0.08391913 0.00580692]
0.9788982744380638 [[-84.83862511]] [0.10223909 0.09947459 0.08391913 0.00580692]
0.9788977638889339 [[-84.83862508]] [0.10223909 0.09947458 0.08391914 0.00580692]
0.9788947504056486 [[-84.8386228]] [0.10223909 0.09947458 0.08391913 0.00580693]
0.9786265477208184 [[-84.85049085]] [0.10222063 0.09954242 0.08397061 0.00580555]
0.9786300240474003 [[-84.85049158]] [0.10222064 0.09954242 0.08397061 0.00580555]
0.9786272438511378 [[-84.85049196]] [0.10222063 0.09954243 0.08397061 0.00580555]
0.9786267422327404 [[-84.85049192]] [0.10222063 0.09954242 0.08397062 0.00580555]
0.9786237343863601 [[-84.85048966]] [0.10222063 0.09954242 0.08397061 0.00580556]
0.9968225227528295 [[-84.85828746]] [0.10220847 0.0995871  0.08400453 0.00580465]
0.9968267028525639 [[-84.85828819]] [0.10220848 0.0995871  0.08400453 0.00580465]
0.996827756662148 [

0.9720186557691931 [[-85.50982844]] [0.09969645 0.10011588 0.09727032 0.00580131]
0.9720165158941024 [[-85.50982768]] [0.09969645 0.10011587 0.09727033 0.00580131]
0.9720144714112359 [[-85.50982636]] [0.09969645 0.10011587 0.09727032 0.00580132]
0.8447236131756327 [[-84.9015175]] [0.10000376 0.10024963 0.10091875 0.00700039]
0.8447248905689309 [[-84.90151807]] [0.10000377 0.10024963 0.10091875 0.00700039]
0.8447258563625525 [[-84.90151841]] [0.10000376 0.10024964 0.10091875 0.00700039]
0.8447237627060717 [[-84.90151752]] [0.10000376 0.10024963 0.10091876 0.00700039]
0.8447214574019513 [[-84.90150861]] [0.10000376 0.10024963 0.10091875 0.0070004 ]
0.9650665121073192 [[-85.50510444]] [0.09971431 0.10012364 0.09748236 0.005871  ]
0.9650678169968934 [[-85.50510504]] [0.09971432 0.10012364 0.09748236 0.005871  ]
0.9650687663486016 [[-85.50510538]] [0.09971431 0.10012365 0.09748236 0.005871  ]
0.9650666291759796 [[-85.50510461]] [0.09971431 0.10012364 0.09748237 0.005871  ]
0.965064568383860

0.8491105485264819 [[-85.46048719]] [0.09982239 0.09919163 0.09985206 0.00582636]
0.8458226373425162 [[-85.46334995]] [0.09980732 0.09922647 0.09990823 0.00582567]
0.8458215110929578 [[-85.46335053]] [0.09980733 0.09922647 0.09990823 0.00582567]
0.8458211539599308 [[-85.46335088]] [0.09980732 0.09922648 0.09990823 0.00582567]
0.8458226639786558 [[-85.46335002]] [0.09980732 0.09922647 0.09990824 0.00582567]
0.8458221806004957 [[-85.46334858]] [0.09980732 0.09922647 0.09990823 0.00582568]
0.8457763390416342 [[-85.46353132]] [0.09980636 0.09922868 0.0999118  0.00582562]
0.8457750809848866 [[-85.4635319]] [0.09980637 0.09922868 0.0999118  0.00582562]
0.84577724189547 [[-85.46353225]] [0.09980636 0.09922869 0.0999118  0.00582562]
0.8457763867765067 [[-85.4635314]] [0.09980636 0.09922868 0.09991181 0.00582562]
0.8457755204327027 [[-85.46352995]] [0.09980636 0.09922868 0.0999118  0.00582563]
0.8457280407357397 [[-85.46350565]] [0.0998065  0.09922837 0.09991129 0.00582563]
0.8457267826691537 [

<ipython-input-12-49e833467f3a>:264: RuntimeWarning: covariance is not positive-semidefinite.
  samples = np.random.multivariate_normal(pred_mu.ravel(), pred_cov, int(nsim))


In [14]:
res2

      fun: array([[-84.617802]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-170.58961248, -241.01056226,   -4.52482283,   91.59013388])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 510
      nit: 26
   status: 0
  success: True
        x: array([0.09980694, 0.09922782, 0.09991054, 0.00582565])

In [11]:
print(samples,trainY)

[[0.05647427 0.01719687 0.04497879 0.03353437 0.02630854 0.00965367
  0.02157527 0.0093279  0.02636699 0.04329367 0.02044637 0.06283334
  0.04499701 0.02893635 0.01379262 0.06272315 0.02062675 0.06285977
  0.02339264 0.07014564 0.01019881 0.01000218 0.05657293 0.03366747
  0.01710609 0.01743097 0.01106156 0.03366732 0.02166039 0.00972314
  0.01714967 0.02656642 0.0205316  0.04494007 0.01361618 0.01746396
  0.02282364 0.01011291 0.0316025 ]] [[0.05035167]
 [0.01216063]
 [0.0447357 ]
 [0.03123432]
 [0.02438436]
 [0.0074955 ]
 [0.01354554]
 [0.00824909]
 [0.02870565]
 [0.04645434]
 [0.02359506]
 [0.05660703]
 [0.04627013]
 [0.02638973]
 [0.00632564]
 [0.04789699]
 [0.01626827]
 [0.07735481]
 [0.01441945]
 [0.05652293]
 [0.01666471]
 [0.01358868]
 [0.05432798]
 [0.03850274]
 [0.01010751]
 [0.01129286]
 [0.009357  ]
 [0.03579652]
 [0.01614062]
 [0.00770211]
 [0.0105933 ]
 [0.03315482]
 [0.01352198]
 [0.05269549]
 [0.00490424]
 [0.00895544]
 [0.01003564]
 [0.01345683]
 [0.02496436]]


In [28]:
trainY

array([[0.05035167],
       [0.01216063],
       [0.0447357 ],
       [0.03123432],
       [0.02438436]])